# Image simulation with dask and generat

The purpose here is to simulate images to identify the best methods for:

- Determining the FLAT image
- Segmenting cells from the background
- Computing the ratio
- Determine the minimal detectable gradient for a given error.

Since subtracting the correct background value is crucial for accurate ratio imaging, we will test the distribution of background values with probplot for normality.

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import scipy
import pandas as pd
import matplotlib.pyplot as plt
import tifffile as tff
import skimage
import skimage.io
import skimage.filters
import zarr
from scipy import ndimage
import dask.array as da
import dask_image
from dask_image import ndfilters
from dask_image import ndmorph

from nima import nima
from nima import utils

store = tff.imread("/home/dati/dt-clop3/data/210920/flatxy.tf8", aszarr=True)

zc1a = zarr.open(store)
zc1a.info

In [ ]:
dd = da.from_zarr(store)
dd

In [ ]:
img = dd[0, 0]
plt.imshow(img, vmax=60)

In [ ]:
bg, bgs, bgfigs = nima.bg(img.compute())

In [ ]:
bg, utils.bg(img.compute())

In [ ]:
plt.figure(figsize=(8, 4))
plt.subplot(1, 2, 1)
plt.hist(bgs, bins=20)
plt.subplot(1, 2, 2)
scipy.stats.probplot(
    bgs,
    plot=plt,
    rvalue=1,
)

In [ ]:
pp = da.mean(
    dask_image.ndfilters.maximum_filter(dd[0:4000:20, 0], size=(100, 1, 1)), axis=0
)

In [ ]:
ppp = pp.compute()

plt.imshow(skimage.filters.gaussian(ppp, 100))

In [ ]:
m = img < skimage.filters.threshold_mean(img)
skimage.filters.threshold_mean((img * m).clip(np.min(img))).compute()

In [ ]:
plt.imshow(img < skimage.filters.threshold_triangle(img))

In [ ]:
from dask_image import ndmorph


def dabg(di):
    m = di < skimage.filters.threshold_mean(di)
    m1 = di < skimage.filters.threshold_mean((di * m).clip(np.min(di)))
    m2 = ndmorph.binary_dilation(~m1)
    return da.ma.masked_array(di, mask=~m1)

In [ ]:
def bg(im):
    m = im < skimage.filters.threshold_mean(im)
    m1 = im < skimage.filters.threshold_mean((im * m).clip(np.min(im)))
    m2 = skimage.morphology.binary_dilation(~m1, footprint=np.ones([1, 1]))
    # m2 = im < skimage.filters.threshold_triangle(np.ma.masked_array(im, mask=~m))
    return np.ma.masked_array(im, mask=m2)

In [ ]:
dabg(img).compute()

In [ ]:
flat = np.ma.mean(dabg(dd[333:500:1, 0]).compute(), axis=0)

In [ ]:
skimage.io.imshow(flat)

### threshold mean clipping to min()

In [ ]:
skimage.io.imshow(bg(dd[0, 0]))

In [ ]:
plt.hist(bg(img).ravel())

In [ ]:
[np.ma.median(bg(dd[i, 0])) for i in range(10)]

In [ ]:
%%time
utils.bg(img.compute(), bgmax=40)

In [ ]:
%%time
np.ma.mean(bg(img.compute()))

In [ ]:
%%time
utils.bg2(img.compute(), bgmax=60)

In [ ]:
utils._bgmax(img.compute(), step=0.1)

In [ ]:
utils.pbar.unregister()

### masked array (ma)

In [ ]:
a = np.ma.masked_array([1, 4, 3], mask=[False, False, True])
b = np.ma.masked_array([10, 2, 6], mask=[False, True, False])

np.ma.median([a, b], axis=0)

In [ ]:
np.ma.median(np.ma.stack([a, b]), axis=0)

In [ ]:
img

In [ ]:
f3 = img > skimage.filters.threshold_local(img.compute(), 601)

In [ ]:
f3

In [ ]:
img[~f3].mean().compute()

In [ ]:
m1 = np.ma.masked_greater(img, 15)

## generat

In [ ]:
image = "bias + noise + dark + flat * (sky + obj)"

In [ ]:
image

- **bias**: generate a wave-like shape along x.
- **noise**: random number will do.
- **dark**: simply a scalar value.
- **flat**: generate some 2D parabolic shape.
- **obj**: circles-ellipsis. (MAYBE: like finite fractals to compare segmentation).
- **sky**: None | some blurred circle-ellipsoid coincident and not with some obj.

fg_prj := 

bg_prj := 

In [ ]:
from nima import generat

In [ ]:
plt.figure(figsize=(12, 2.8))
plt.subplot(1, 5, 1)
plt.title("BIAS")
skimage.io.imshow(generat.gen_bias(64, 64))
plt.subplot(1, 5, 2)
plt.title("FLAT")
skimage.io.imshow(generat.gen_flat(64, 64))
plt.subplot(1, 5, 3)
plt.title("Object")
skimage.io.imshow(generat.gen_object(64, 64, max_radius=7))
plt.subplot(1, 5, 4)
plt.title("OBJS")
skimage.io.imshow(
    generat.gen_objs(
        100,
        30,
        max_radius=12,
        min_radius=6,
    )
)

In [ ]:
objs = generat.gen_objs(15, 20, max_radius=12, min_radius=6, ncols=64, nrows=64)
frame = generat.gen_frame(objs, None, None, dark=10, sky=0, noise_sd=6)

bg, bgs, bgfigs = nima.bg(frame.astype("float"))
plt.hist(bgs, bins=8)
bg

In [ ]:
bg_arcsinh = []
bg_entropy = []
bg_adaptive = []
bg_liadaptive = []
bg_lili = []
bg_utils = []
bg2_utils = []

for _ in range(25):
    objs = generat.gen_objs(150, 60, max_radius=12, min_radius=6, ncols=64, nrows=64)
    frame = generat.gen_frame(objs, None, None, dark=10, sky=0, noise_sd=2)
    bg_arcsinh.append(nima.bg(frame.astype("float"), kind="arcsinh")[0])
    # bg_entropy.append(nima.bg(frame, kind='entropy')[0])
    bg_adaptive.append(nima.bg(frame, kind="adaptive")[0])
    bg_liadaptive.append(nima.bg(frame, kind="li_adaptive")[0])
    bg_lili.append(nima.bg(frame, kind="li_li")[0])
    bg_utils.append(utils.bg(frame)[0])
    bg2_utils.append(utils.bg2(frame)[0])

In [ ]:
skimage.io.imshow(frame)

In [ ]:
# Create DataFrame to organize results and plot boxplot
df = pd.DataFrame(
    np.column_stack(
        #        (bg_arcsinh, bg_entropy, bg_adaptive, bg_liadaptive, bg_lili, bg_utils)
        (bg_arcsinh, bg_adaptive, bg_liadaptive, bg_lili, bg_utils, bg2_utils)
    ),
    columns=["arcsinh", "adaptive", "li_adaptive", "li li", "utils.bg", "utils.bg2"],
)
f = df.boxplot(vert=False, showfliers=False)

In [ ]:
import seaborn as sb

sb.regplot(pd.DataFrame(dict(x=bg_arcsinh, y=bg2_utils)), x="x", y="y")

In [ ]:
np.argmax(bg_arcsinh)

In [ ]:
bg2_utils.pop(14)

In [ ]:
def gen_object(
    nrows: int = 128, ncols: int = 128, min_radius: int = 6, max_radius: int = 12
):
    """Generate a single small object without random positioning."""
    x_idx, y_idx = np.indices((nrows, ncols))
    x_obj = nrows // 2  # Center of the frame
    y_obj = ncols // 2  # Center of the frame
    radius = np.random.randint(min_radius, max_radius)
    ellipsis = np.random.rand() * 3.5 - 1.75
    mask = np.array(
        (x_idx - x_obj) ** 2
        + (y_idx - y_obj) ** 2
        + ellipsis * (x_idx - x_obj) * (y_idx - y_obj)
        < radius**2
    )
    return mask


# Generate a single small object
small_object = gen_object(nrows=12, ncols=12, min_radius=3, max_radius=4)

# Plot the generated object
plt.imshow(small_object, cmap="gray")
plt.title("Single Small Object")
plt.xlabel("X")
plt.ylabel("Y")
plt.show()

In [ ]:
import scipy.signal

# Convolve the small object with the flat image
convolved_image = scipy.signal.convolve2d(flat, small_object, mode="same")

# Plot the convolved image
plt.imshow(convolved_image, cmap="gray")
plt.colorbar()
plt.title("Convolved Image")

In [ ]:
flat.shape[1] - small_object.shape[1]

In [ ]:
# Number of frames in the stack
num_frames = 10000

# Initialize an empty stack to store the frames
stack = np.zeros_like(flat)

# Iterate over each frame in the stack
for _ in range(num_frames):
    # Generate random coordinates for the position of the small object within the flat image
    x_pos = np.random.randint(0, flat.shape[1] - small_object.shape[1])
    y_pos = np.random.randint(0, flat.shape[0] - small_object.shape[0])

    # Add the small object to the flat image at the random position
    flat_image_with_object = flat.copy()
    flat_image_with_object[
        y_pos : y_pos + small_object.shape[0], x_pos : x_pos + small_object.shape[1]
    ] += small_object

    # Add the frame with the small object to the stack
    stack += flat_image_with_object

# Plot the summed stack
estimated = stack / stack.mean()
plt.imshow(estimated, cmap="gray")
plt.colorbar()
plt.title("Summed Stack with Small Object")
plt.show()
# plt.imshow(estimated - flat, cmap='gray')
skimage.io.imshow(ndimage.gaussian_filter(estimated, sigma=3) - flat)

In [ ]:
# Calculate the Fourier transform of the small object
fourier_transform_obj = np.fft.fft2(small_object)

# Calculate the magnitude spectrum of the Fourier transform
magnitude_spectrum = np.abs(np.fft.fftshift(fourier_transform_obj))

# Plot the magnitude spectrum
plt.imshow(magnitude_spectrum, cmap="gray")
plt.colorbar(label="Magnitude")
plt.title("Magnitude Spectrum of Fourier Transform")
plt.xlabel("Frequency (kx)")
plt.ylabel("Frequency (ky)")
plt.show()

In [ ]:
# Apply the convolution theorem
flat_fft = np.fft.fft2(stack)

# Calculate the Fourier transform of the small object
fourier_transform_obj = np.fft.fft2(small_object)

# Pad the small object to match the shape of flat
padded_obj = np.pad(
    small_object,
    (
        (0, flat.shape[0] - small_object.shape[0]),
        (0, flat.shape[1] - small_object.shape[1]),
    ),
    mode="constant",
)

# Calculate the Fourier transform of the padded small object
fourier_transform_padded_obj = np.fft.fft2(padded_obj)

# Calculate the Fourier transform of the flat image
flat_fft = np.fft.fft2(flat)

# Perform element-wise division
result_fft = np.fft.ifftshift(
    np.fft.ifft2(np.fft.fftshift(flat_fft / fourier_transform_padded_obj))
)
# result_fft = np.fft.ifftshift(np.fft.ifft2(np.fft.fftshift(flat_fft / fourier_transform_obj)))

# Take the real part to get rid of any numerical artifacts
result = np.real(result_fft)

# Plot the resulting flat image
plt.imshow(result, cmap="gray")
plt.colorbar(label="Intensity")
plt.title("Resulting Flat Image")
plt.xlabel("X")
plt.ylabel("Y")
plt.show()

In [ ]:
from nima import generat

flat = generat.gen_flat()
bias = np.zeros((128, 128))

objs = generat.gen_objs(max_fluor=20, max_n_obj=80)
frame = generat.gen_frame(objs, bias=bias, flat=flat, noise_sd=2, dark=7, sky=7)

# Plot the frame
plt.imshow(frame, cmap="viridis", origin="lower")
plt.colorbar(label="Intensity")
plt.title("Simulated Image Frame without Bias")
plt.xlabel("X")
plt.ylabel("Y")
plt.show()

In [ ]:
from tqdm import tqdm

# Generate a stack of frames
num_frames = 100
frame_stack = []
for _ in tqdm(range(num_frames), desc="Generating Frames"):
    objs = generat.gen_objs(max_fluor=20, max_n_obj=80)
    frame = generat.gen_frame(objs, bias=bias, flat=flat, noise_sd=2, dark=7, sky=7)
    frame_stack.append(frame)

In [ ]:
from functools import partial

p999 = partial(np.percentile, q=99.7)
p999.__name__ = "percentile 99.9%"


def diff_plot(im, flat, title):
    f, axs = plt.subplots(1, 2)
    diff = im / im.mean() - flat
    skimage.io.imshow(diff, ax=axs[0])
    axs[1].hist(diff.ravel())
    f.suptitle(title)
    return diff.mean(), diff.std()


def prj_plot(t_prj, title, sigma=128 / 11):
    im = ndimage.gaussian_filter(t_prj, sigma=sigma)
    return diff_plot(im, flat, title)


def prj(stack, func, sigma):
    t_prj = func(stack, axis=0)
    return prj_plot(t_prj, func.__name__)


prj(frame_stack, np.max, sigma=3)
prj(frame_stack, p999, sigma=3)
prj(frame_stack, np.mean, sigma=3)
prj(frame_stack, np.median, sigma=3)
prj(frame_stack, np.min, sigma=3)

In [ ]:
objs = generat.gen_objs(max_fluor=20, max_n_obj=8)
frame = generat.gen_frame(objs, bias=bias, flat)
plt.imshow(frame)

In [ ]:
bias = np.zeros((128, 128))
flat = np.ones((128, 128))

stack = np.stack(
    [
        generat.gen_frame(
            generat.gen_objs(max_fluor=10), bias, flat, noise_sd=10, sky=10
        )
        for i in range(1000)
    ]
)

In [ ]:
stat_bg = []
for s in stack[:]:
    stat_bg.append(utils.bg(s)[0])

In [ ]:
plt.hist(stat_bg)
np.mean(stat_bg), np.std(stat_bg)

bg2 was less robust with small signal

## what is the best projection for flat calculation? 

In [ ]:
bias = generat.gen_bias()
flat = generat.gen_flat()
stack = np.stack(
    [
        generat.gen_frame(generat.gen_objs(max_fluor=20), bias, flat, noise_sd=1, sky=2)
        for i in range(1000)
    ]
)

In [ ]:
def splot(stack, num=4):
    f, axs = plt.subplots(1, num)
    for i in range(num):
        axs[i].imshow(stack[np.random.randint(len(stack))])


splot(stack)

In [ ]:
def diff_plot(im, flat, title):
    f, axs = plt.subplots(1, 2)
    diff = im / im.mean() - flat
    skimage.io.imshow(diff, ax=axs[0])
    axs[1].hist(diff.ravel())
    f.suptitle(title)
    return diff.mean(), diff.std()


def prj_plot(t_prj, title, sigma=128 / 11):
    im = ndimage.gaussian_filter(t_prj, sigma=sigma)
    return diff_plot(im, flat, title)


def prj(stack, func):
    t_prj = func(stack, axis=0)
    return prj_plot(t_prj, func.__name__)


prj(stack, np.max)

In [ ]:
prj(stack, np.mean)

In [ ]:
prj(stack, np.median)

In [ ]:
from functools import partial

p999 = partial(np.percentile, q=99.9)
p999.__name__ = "percentile 99.9%"

prj(stack, p999)

In [ ]:
im = np.mean(
    ndfilters.median_filter(
        da.from_array(stack[:100] - bias), size=(32, 16, 16)
    ).compute(),
    axis=0,
)
prj_plot(im, "dd", sigma=7)

#### Knowing the Bias.

In [ ]:
prj(stack - bias, p999)

In [ ]:
prj(stack - bias, np.mean)

### Using fg and bg masks?

And assuming we know the bias of the camera.

In [ ]:
def mask_plane(plane, bg_ave=2, bg_std=1.19, erf_pvalue=0.01):
    p = utils.prob(plane, bg_ave, bg_std)
    p = ndimage.median_filter(p, size=2)
    mask = p > erf_pvalue
    mask = skimage.morphology.remove_small_holes(mask)
    return np.ma.masked_array(plane, mask=~mask), np.ma.masked_array(plane, mask=mask)


plt.imshow(mask_plane(stack[113], *utils.bg(stack[113]))[0])

In [ ]:
bgs, fgs = list(zip(*[mask_plane(s - bias, *utils.bg(s - bias)) for s in stack]))

splot(bgs)

In [ ]:
t_prj = np.ma.mean(np.ma.stack(bgs), axis=0)
prj_plot(t_prj, "Bg mean", sigma=3)

In [ ]:
t_prj = np.ma.max(np.ma.stack(fgs), axis=0)
prj_plot(t_prj, "Fg max (bias known)", sigma=2)

In [ ]:
bgs, fgs = list(zip(*[mask_plane(s, *utils.bg(s)) for s in stack]))

bg_prj1 = np.ma.mean(np.ma.stack(bgs[:]), axis=0)
fg_prj1 = np.ma.mean(np.ma.stack(fgs[:]), axis=0)
im = fg_prj1 - bg_prj1
diff_plot(ndimage.gaussian_filter(im, 1), flat, "Bg mean - fg mean")

In [ ]:
bg_prj = np.ma.mean(bgs, axis=0)
fg_prj = np.ma.max(fgs, axis=0)
# im = ndimage.median_filter(bg_prj-fg_prj, size=60) #- 2 * flat
im = ndimage.gaussian_filter(bg_prj - fg_prj, sigma=14)  # - 2 * flat

diff_plot(im, flat, "m")

In [ ]:
t_prj = np.ma.max(fgs, axis=0)
prj_plot(t_prj, "Fg MAX", sigma=13)

In [ ]:
eflat = bg_prj - fg_prj
eflat /= eflat.mean()
eflat = ndimage.gaussian_filter(eflat, sigma=13)

diff_plot(eflat, flat, "eflat")

## When bias and flat are unknown...

- bias = bg_prj - sky * flat
- bias = fg_prj - flat

sky * flat - flat = bg_prj - fg_prj

In [ ]:
diff_plot((bg_prj1 - bias) / 2, flat, "")

In [ ]:
plt.imshow((im - bias) / (im - bias).mean() - flat)
plt.colorbar()

## cfr. nima.bg

In [ ]:
# r = nima.bg((stack[113] - bias) / flat)
r = nima.bg(stack[111])

In [ ]:
r[1].mean(), r[1].std()

In [ ]:
utils.bg(stack[111])

In [ ]:
bias.mean() + 2

## geometric mean

In [ ]:
vals = [0.8, 0.1, 0.3, 0.1, 0.8, 0.8, 0.8, 0.1, 0.8]

np.median(vals), scipy.stats.gmean(vals), np.mean(vals)

In [ ]:
(0.8 * 0.8 * 0.8 * 0.8 * 0.1) ** (1 / 5)